In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

import numpy as np # linear algebra
import scipy.sparse as sps
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import surprise as sp
import os
from sklearn.preprocessing import normalize
import cmath
from numpy.linalg import norm
import surprise as sp
from sklearn.metrics.pairwise import cosine_similarity

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

def read_data(filename):
    dataset = pd.read_csv(filename)
    return dataset


category_tree.csv
events.csv
item_properties_part1.csv
item_properties_part2.csv
rating_matrix.csv



In [2]:
# Read the dataset
category_tree = read_data("../input/category_tree.csv")
events = read_data("../input/events.csv")
item_1 = read_data("../input/item_properties_part1.csv")
item_2 = read_data("../input/item_properties_part2.csv")

print("Read all data in pandas object complete")

Read all data in pandas object complete


In [117]:
# PRE PROCESSING
# combine item matrix
import math
item_properties = pd.concat(([item_1, item_2]), ignore_index=True)
events_full = events
test, train = pd.DataFrame([]), pd.DataFrame([])
n = events.shape[0]

# use only transaction data for now
events = events[events.event=='transaction']
events = events.sort_values('timestamp')
events = events[events.groupby('visitorid').visitorid.transform(len) > 1]

#events = events.head(1000)
users = sorted(events.visitorid.unique())
items = sorted(item_properties.itemid.unique())
groupby_users = events.groupby('visitorid')

counts = events.groupby('visitorid').count()['timestamp'].tolist()
ind = 0

for name, group in events.groupby('visitorid'):
    train_size = int(math.ceil(0.7*n))
    test_size = int(math.floor(0.3*n))
    if test_size == 0:
        test_size = 1
    train = train.append(group.head(train_size))
    test = test.append(group.tail(test_size))
    ind +=1
print('Train and test constructed with sizes',train.shape[0],test.shape[0])
# create a user-item interaction matrix
rows, cols = max(events.visitorid.unique()), max(item_properties.itemid.unique())
print(rows,cols)
ui_matrix = sps.lil_matrix((rows+2, cols+2), dtype=int) # empty matrix
sampled_train = sps.lil_matrix((rows+2, cols+2), dtype=int) # empty matrix

for index, row in train.iterrows():
    if row['event'] == 'transaction':
        ui_matrix[row['visitorid'],row['itemid']] = 1
print('here 1')
sample = []
row_sums = ui_matrix[users,:].sum(axis=1)
#col_sums = ui_matrix[:,items].sum(axis=0)
sample = [i for i in range(len(row_sums)) if row_sums[i] >=5]
#sample_items = [i for i in range(len(col_sums)) if col_sums[i] >=1]
print('here 2')
sampled_train = []
for user_index in sample:
    sampled_train.append(list(ui_matrix[users[user_index],:].A[0]))
print('Sampled_train constructed, contains item purchases (5+ items per row)')

#ui_matrix = normalize(ui_matrix, norm='l1', axis=1, copy=False)
#ui_matrix *= 5

#A = pd.DataFrame(columns=['userID', 'itemID', 'rating'])
#R,C = sampled_train.nonzero()
#contents = sampled_train[R,C].toarray()
#A['userID'] = pd.Series(R)
#A['itemID'] = pd.Series(C)
#A['rating'] = pd.Series(contents[0])

('Train and test constructed with sizes', 13314, 13314)
(1406087, 466866)
here 1
here 2
Sampled_train constructed, contains item purchases (5+ items per row)


In [118]:
len(events.visitorid.unique()),len(test.visitorid.unique())

(2576, 2576)

In [91]:
len(users)

11719

In [119]:
# LINK ANALYSIS
print('Commence link analysis')

gamma = 0.9
A = sampled_train
B = []
lol = []


for i,user in enumerate(sample):
    B.append(A[i]/row_sums[user,0]**gamma)
A = np.array(A)
B = np.array(B)
CR_prev=np.identity(len(sample))
CR_0=CR_prev
counter = 0
print('Begin loop')
while(True):
    PR = np.dot(A.T,CR_prev)
    CR_new = np.dot(B,PR)
    r_s = CR_new.sum(axis=1)
    new_matrix = CR_new / r_s[:, np.newaxis]   
    CR_new = new_matrix+CR_0
    if(np.linalg.norm(CR_new-CR_prev))<=10:
        break
    CR_prev = CR_new
    counter += 1

print('Link Analysis Done')
PR=PR.T


Commence link analysis
Begin loop
Link Analysis Done


In [120]:
#Get recommendations
rec = []
k = 5
#PR -> NxM (M-users, N-items) 
#Get top k items columnwise
for arr in PR:
    rec.append(arr.argsort()[-k:][::-1])


In [126]:
# Get ground truth using future transactions that exist in test
user_dict = {}
for i,v in enumerate(sample):
    if users[v] not in user_dict:
        user_dict[users[v]]=i
print('Dict done')
test_users = test['visitorid']
hits = np.zeros((max(users))+1)
for ind,row in test.iterrows():
    tu = row['visitorid']
    if tu in user_dict:
        algo_rec = rec[user_dict[tu]]
        if row['itemid'] in algo_rec:
            hits[tu]+=1


Dict done


In [ ]:
#User Based
WC = 

In [125]:
len(hits)

1406087

In [144]:
#Evaluation Metrics

#Precision
precision = hits/k
P_c = precision[test.sort_values('visitorid').visitorid.unique()]
no_of_test_users = len(test.visitorid.unique())
avg_precision = sum(P_c)/no_of_test_users
print(avg_precision)

#Recall
no_interactions = list(test.sort_values('visitorid').groupby('visitorid').count().event)
recall = hits[list(test.sort_values('visitorid').visitorid.unique())]/no_interactions
R_c = recall
avg_recall = sum(R_c)/no_of_test_users
print(avg_recall)

#F-measure
F = 2*P_c*R_c/(P_c+R_c)
nan_index = np.isnan(F)
F[nan_index] = 0
avg_F = sum(F)/len(F)
print(avg_F)


0.144099378882
0.0754860656057
0.0894631818492


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in divide


In [79]:
len(train.visitorid.unique())

2576

1407398

In [5]:
#Check sparsity of the matrix
sparsity = 100*(1 - float( len(C) )/(len(events.visitorid.unique())*len(item_properties.itemid.unique())))
sparsity
#len(R)

NameError: name 'C' is not defined